In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1931, -0.3018,  0.0218, -0.1668,  0.0892, -0.0308,  0.3854, -0.0073,
          0.0974,  0.1060],
        [-0.1319, -0.1789,  0.1475, -0.1228,  0.0437, -0.1521,  0.4561, -0.0638,
          0.1990,  0.0585]], grad_fn=<AddmmBackward>)

# 自定义块

In [3]:
class MLP(nn.Module):
    # 用模型参数声明层，这里，我们声明两个全连接的层
    def __init__(self):
        # 调用`MLP`的父类`Block`的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数`params`（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层
    
    # 定义模型的正向传播，即，如何根据输入 X 返回所需的模型输出
    def forward(self, X):
        # 注意， 这里我们使用的ReLU函数版本，其在nn.functional模块中定义
        return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
net(X)

tensor([[ 0.0975, -0.0095,  0.1254, -0.1873, -0.1897, -0.2213,  0.0277,  0.0960,
          0.4260,  0.1234],
        [-0.0751, -0.1775,  0.2706, -0.2853, -0.1730,  0.0069, -0.0148,  0.2367,
          0.3481,  0.0682]], grad_fn=<AddmmBackward>)

# 顺序块

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            # 这里，block 是 Module 子类的一个实例，我们把它保存在 Module 类的成员变量
            # _children 中， block 的类型是OrderedDict
            self._modules[block] = block 
    
    def forward(self, X):
        # OrderedDict 保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [7]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1659, -0.1755, -0.1523,  0.0943,  0.0926, -0.0754, -0.2288, -0.0595,
          0.0196, -0.0292],
        [-0.0497, -0.1370, -0.0319, -0.0830,  0.1497, -0.0450, -0.2233,  0.0117,
         -0.0215,  0.0154]], grad_fn=<AddmmBackward>)

# 在正向传播函数中执行代码

In [14]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数，因此在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及 relu 和 dot 函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层，这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2 
        return X.sum()

In [15]:
net = FixedHiddenMLP()
net(X)

tensor(-0.1347, grad_fn=<SumBackward0>)

In [16]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1828, grad_fn=<SumBackward0>)

# 效率